In [ ]:
import numpy as np

class mylasso():
    def __init__(self, lamcoef = 0.1, max_iter=1000, tol=1e-6, const_regu = False):
        """lamcoef: the regularization coefficient
           max_iter: maximum number of iteration for model training
           tol: tolerance for the stopping criteria for model training
           const_regu: whether the constant term should be regularized, default to False
           """
        ### Add your code here ###
        self.lamcoef = lamcoef
        self.max_iter = max_iter
        self.tol = tol #當L的變化跟上一輪比較，相差小於tol就停下來
        self.const_regu = const_regu

    def fit(self, x_train, y_train, winit = "ridge", keep_traindata = True, verbose = False):
        """Fit a Lasso model
           x_train: a numpy matrix that contains training features
           y_train: a numpy matrix that contains the label
           winit: how weights are initialized, default to "ridge", ridge regression
           keep_traindata: whether the object is going to keep training data after the training process completed
           verbose: output a lot of message"""
        ### Add your code here ###
        self.x_train = x_train
        self.y_train = y_train
        self.winit = winit
        self.keep_traindata = keep_traindata
        self.verbose = verbose
        self.M = len(x_train[0])
        self.N = len(x_train)
        self.bestW = np.zeros(self.M+1)
        
        x_train_1 = np.insert(x_train,0,1,axis=1) #加入常數項
        
        # 先用ridge預設一組 W
        self.InitW(3)
        self.W = np.insert(self.W,0,self.b,axis=0) #把ridge得到的 W 跟常數相拼在一起
        
        #print(self.W)
        
        # 再開始train
        count_iter = 0
        cau_2 = self.N*self.lamcoef #因為都一樣所以放外面減少計算量
        
        minL = ((np.sum((y_train-np.dot(x_train_1,self.W))**2))/2) + (self.lamcoef*np.sum(np.absolute(self.W))) 
        
        while count_iter<=self.max_iter: #當更新輪數大於max_iter就停止
            j = count_iter%len(self.W) #讓count_iter超過W的長度時可以再輪回頭更新, j表示該輪要更新Wj
            for i in range(N):
                if(i==j):
                    continue
                else:
                    cau_1 = 0.0
                    cau_3 = 0.0
                    cau_0 = (np.dot(self.W.T, x_train_1[i].T))-(self.W[j]*x_train_1[i][j])
                    cau_1 += (self.y_train[i]-cau_0)*(x_train_1[i][j])
                    cau_3 += ((x_train_1[i][j])**2)+1                  
            if (cau_1-cau_2)/cau_3 >0:
                self.W[j]=(cau_1-cau_2)/cau_3
            elif (cau_1+cau_2)/cau_3 <0:
                self.W[j]=(cau_1+cau_2)/cau_3
            else:
                self.W[j]=0
            #print(W[j])
            
            ### 每輪結束確認此次 W 算出來的 Loss ###
            # Training Loss

            L = ((np.sum((y_train-np.dot(x_train_1,self.W))**2))/2) + (self.lamcoef*np.sum(np.absolute(self.W))) 
            print(L)
            if L<minL: # 要存下 L 最小的 W
                minL = L
                self.bestW = self.W
                # print(minL)
                # print(self.bestW)
            
            count_iter +=1
        
        ### 列出題目規定的資料 ###
        self.y_pred = np.dot(x_train_1, self.bestW)
        print("RMSE:",self.RMSE())
        print("MAE:",self.MAE())
        print("Intercept:",self.W[0]) 
        print("FeatureWeights:",self.W[1:]) 
        
        #非零權重個數
        
    def InitW(self, iterations): #用 Ridge 找一組初始化的 W
        self.iteration = iterations
        self.b = 0 #w0
        self.W = np.zeros(self.M)
        for i in range(self.iteration):
            y_pred = np.dot(self.x_train,self.W)+self.b
            update_W = (-(2*np.dot((self.x_train.T),(self.y_train-y_pred))) +               
               (2*self.lamcoef*self.W)) / self.N      
            update_b = -2*np.sum(self.y_train-y_pred )/self.N  
            # update    
            self.W = self.W - update_W     
            self.b = self.b - update_b
            #print(y_pred)
    
    def RMSE(self):
        MSE = np.sum((self.y_pred-self.y_train)**2)
        return (MSE/self.N)**0.5
        
    def MAE(self):
        return np.sum(np.absolute(self.y_pred-self.y_train))/self.N
                    
            
    def predict(self, x_test):
        """Make prediction using trained model"""
        
        ### Add your code here ###
        W = self.W #讀train好的W
        x_test_1 = np.insert(x_test,0,1,axis=1) #補上常數項
        y_pred = np.dot(x_test_1, W)
        return y_pred

In [ ]:
lasso = mylasso()
lasso.fit(X_train_sd, Y_train)
ypred_1 = lasso.predict(X_test)
#evaluate_1[i] = RMSE(ypred_1,Y_test)
print(ypred_1)